# Quantum Volume

In [2]:
from qiskit_experiments.library import QuantumVolume
from qiskit_experiments.framework import BatchExperiment
from qiskit import transpile, execute
from qiskit.circuit.library import QuantumVolume as QuantumVolumeCircuit
from qiskit.quantum_info import Statevector

from huayi_providers.fake_huayi12 import FakeHuayi12, FakeHuayi12V2
from qiskit_ionq import IonQProvider
from qiskit.providers.fake_provider import *
from qiskit_aer.noise.noise_model import NoiseModel

from qiskit import Aer
from qiskit_aer import AerSimulator

import pandas as pd

## Use qiskit-experiments built-in QV measurement

**Error message**

Don't use V1 backend (including FakeHuayi12, FakeMontreal, ...)

something wrong with:

``transpiled_circuits = experiment._transpiled_circuits()``


There is also an unknown problem when n_qubits or trials is too big when use V2 backend

some discussions:

https://github.com/Qiskit-Extensions/qiskit-experiments/issues/846


In [ ]:
def collect_data(batch_data):
    results_table = pd.DataFrame()
    for i in range(batch_data.num_experiments):
        qv_result = batch_data.child_data(i).analysis_results("quantum_volume")
        data = {'QV': qv_result.value} | {'quality': qv_result.quality} | qv_result.extra
        results_table = pd.concat([results_table, pd.DataFrame(data, index=[i])], ignore_index=True)
    return results_table

In [ ]:
simulator_huayi = FakeHuayi12V2()
simulator_montreal = FakeMontrealV2()

# Each QuantumVolume() is an Experimet class
# The length of tuple() is the circuit depth
exps = [QuantumVolume(tuple(range(i)), trials=100) for i in range(3,7)]

# Batch the experiments (with different depth) into one object
batch_exp = BatchExperiment(exps)
batch_exp.set_transpile_options(optimization_level=3)

# Run with the specified backend
data_huayi = batch_exp.run(simulator_huayi).block_for_results()
data_montreal = batch_exp.run(simulator_montreal).block_for_results()

# Collect the results in the 'results_table'
results_huayi = collect_data(data_huayi)
results_montreal = collect_data(data_montreal)

# results_table = pd.DataFrame()
# for i in range(batch_exp.num_experiments):
#     qv_result = data_huayi.child_data(i).analysis_results("quantum_volume")
#     data = {'QV': qv_result.value} | {'quality': qv_result.quality} | qv_result.extra
#     results_table = pd.concat([results_table, pd.DataFrame(data, index=[i])], ignore_index=True)

display(results_huayi)
display(data_huayi.child_data(batch_exp.num_experiments-1).figure(0))

display(results_montreal)
display(data_montreal.child_data(batch_exp.num_experiments-1).figure(0))

In [ ]:
# Single run

# Each QuantumVolume() is an Experimet class
# The length of tuple() is the circuit depth
exps = QuantumVolume(tuple(range(4)), trials=160) 

# Batch the experiments (with different depth) into one object
exps.set_transpile_options(optimization_level=3)

backend = AerSimulator.from_backend(FakeMontrealV2())
# Run with the specified backend
expdata = exps.run(backend).block_for_results()

expdata.figure(0)

## From qiskit textbook

https://github.com/Qiskit/textbook/blob/main/notebooks/quantum-hardware/measuring-quantum-volume.ipynb


In [3]:
def get_heavy_outputs(counts):
    """Extract heavy outputs from counts dict.
    Args:
        counts (dict): Output of `.get_counts()`
    Returns:
        list: All states with measurement probability greater
              than the mean.
    """
    # sort the keys of `counts` by value of counts.get(key)
    sorted_counts = sorted(counts.keys(), key=counts.get)
    # discard results with probability < median
    heavy_outputs = sorted_counts[len(sorted_counts)//2:]
    return heavy_outputs

def check_threshold(nheavies, ncircuits, nshots):
    """Evaluate adjusted threshold inequality for quantum volume.
    Args:
        nheavies (int): Total number of heavy outputs measured from device
        ncircuits (int): Number of different square circuits run on device
        nshots (int): Number of shots per circuit
    Returns:
        Bool:
            True if heavy output probability is > 2/3 with 97% certainty,
            otherwise False
    """
    from numpy import sqrt
    numerator = nheavies - 2*sqrt(nheavies*(nshots-(nheavies/ncircuits)))
    return bool(numerator/(ncircuits*nshots) > 2/3)


def test_qv(device, nqubits, ncircuits, nshots):
    """Try to achieve 2**nqubits quantum volume on device.
    Args:
        device (qiskit.providers.Backend): Device to test.
        nqubits (int): Number of qubits to use for test.
        ncircuits (int): Number of different circuits to run on the device.
        nshots (int): Number of shots per circuit.
    Returns:
        Bool
            True if device passes test, otherwise False.
    """
    def get_ideal_probabilities(circuit):
        """Simulates circuit behaviour on a device with no errors."""
        state_vector = Statevector.from_instruction(
                circuit.remove_final_measurements(inplace=False)
            )
        return state_vector.probabilities_dict()

    def get_real_counts(circuit, backend, shots):
        """Runs circuit on device and returns counts dict."""
        t_circuit = transpile(circuit, backend)
        job = backend.run(t_circuit,
                          shots=shots,
                          memory=True)
        return job.result().get_counts()

    # generate set of random circuits
    qv_circuits = [
        QuantumVolumeCircuit(nqubits) for c in range(ncircuits)
    ]

    nheavies = 0  # number of measured heavy outputs
    for circuit in qv_circuits:
        # simulate circuit
        ideal_heavy_outputs = get_heavy_outputs(
            get_ideal_probabilities(circuit)
        )
        # run circuit on device
        circuit.measure_all()
        real_counts = get_real_counts(circuit, device, nshots)
        # record whether device result is in the heavy outputs
        for output, count in real_counts.items():
            if output in ideal_heavy_outputs:
                nheavies += count

    # do statistical check to see if device passes test
    is_pass = check_threshold(nheavies, ncircuits, nshots)
    # calculate percentage of measurements that are heavy outputs
    percent_heavy_outputs = nheavies*100/(ncircuits * nshots)

    results = {
        "backend":device.name(),
        "n_qubits":nqubits,
        "QV": 2**nqubits, 
        "HOP": percent_heavy_outputs, 
        "success": is_pass, 
        "n_circuits": ncircuits,
        "n_shots": nshots}

    print(f"Quantum Volume: {2**nqubits}\n"
          f"Percentage Heavy Outputs: {percent_heavy_outputs:.1f}%\n"
          f"Passed?: {is_pass}\n")
    return results


In [4]:
results_huayi = pd.DataFrame()
results_montreal = pd.DataFrame()

for nqubits in range(3,10):
    result = test_qv(FakeHuayi12(), nqubits, ncircuits=200, nshots=50)
    results_huayi = pd.concat([results_huayi, pd.DataFrame(result, index=[0])], ignore_index=True)
    
    result = test_qv(FakeMontreal(), nqubits, ncircuits=200, nshots=50)
    results_montreal = pd.concat([results_montreal, pd.DataFrame(result, index=[0])], ignore_index=True)

display(results_huayi)
display(results_montreal)

Quantum Volume: 8
Percentage Heavy Outputs: 81.0%
Passed?: True

Quantum Volume: 8
Percentage Heavy Outputs: 78.1%
Passed?: True

Quantum Volume: 16
Percentage Heavy Outputs: 82.0%
Passed?: True

Quantum Volume: 16
Percentage Heavy Outputs: 75.7%
Passed?: True

Quantum Volume: 32
Percentage Heavy Outputs: 84.5%
Passed?: True

Quantum Volume: 32
Percentage Heavy Outputs: 76.8%
Passed?: True

Quantum Volume: 64
Percentage Heavy Outputs: 82.0%
Passed?: True

Quantum Volume: 64
Percentage Heavy Outputs: 67.9%
Passed?: False

Quantum Volume: 128
Percentage Heavy Outputs: 81.0%
Passed?: True

Quantum Volume: 128
Percentage Heavy Outputs: 66.1%
Passed?: False

Quantum Volume: 256
Percentage Heavy Outputs: 79.2%
Passed?: True

Quantum Volume: 256
Percentage Heavy Outputs: 58.2%
Passed?: False

Quantum Volume: 512
Percentage Heavy Outputs: 78.2%
Passed?: True

Quantum Volume: 512
Percentage Heavy Outputs: 55.6%
Passed?: False



,backend,n_qubits,QV,HOP,success,n_circuits,n_shots
0,fake_huayi12,3,8,80.98,True,200,50
1,fake_huayi12,4,16,82.04,True,200,50
2,fake_huayi12,5,32,84.52,True,200,50
3,fake_huayi12,6,64,82.01,True,200,50
4,fake_huayi12,7,128,80.97,True,200,50
5,fake_huayi12,8,256,79.18,True,200,50
6,fake_huayi12,9,512,78.23,True,200,50


,backend,n_qubits,QV,HOP,success,n_circuits,n_shots
0,fake_montreal,3,8,78.13,True,200,50
1,fake_montreal,4,16,75.74,True,200,50
2,fake_montreal,5,32,76.79,True,200,50
3,fake_montreal,6,64,67.94,False,200,50
4,fake_montreal,7,128,66.07,False,200,50
5,fake_montreal,8,256,58.25,False,200,50
6,fake_montreal,9,512,55.59,False,200,50
